# Problem Set 5b: Creating a Decision RNN (80 pts)

In this assignment, you will perform a simplified variant of the RNN analyses in Mante*, Sussillo* et al., Nature, 2013 (MS2013). You will train an RNN to perform evidence accumulation (a.k.a. integration), and visualize various features of your trained RNN. Specifically, you will train an RNN to output the running sum of a time-series stimulus input, mimicking the evidence accumulation computation that is central to many simple decision-making tasks. You will visualize the output of the trained RNN on one trial for each stimulus strength, and you will display the condition-averaged activity of a few of the RNN's hidden units. You will then visualize low-dimensional representations of the high-dimensional hidden states of the RNN using PCA and a regression technique akin to Targeted Dimensionality Reduction from MS2013. These visualization techniques are a first step toward understanding how the RNN is representing information and performing computation.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(42)

### Part 1: Create the Synthetic Data (5 pts)
First, create the data (inputs and target outputs) used to train the RNN. 

For the inputs, you will generate synthetic time-series stimuli, which represents the time-varying random dots stimuli from MS2013. The stimulus will be two-dimensional, one representing motion energy and the other representing the average color. For each of these two dimensions, you will generate time-series stimuli from univariate Gaussian distributions whose mean represents the corresponding stimulus strength. In a neuroscience experiment, these stimulus strengths might be referred to as "conditions" and would allow the experimenter to control the difficulty of each trial.

Draw 40 trials of 25-timestep stimuli for each condition (200 trials total) from unit-variance Gaussians with condition-specific means of {-1, -0.5, 0, 0.5, 1}. Each time step should be independent. You can use the `torch.randn` function to generate samples from a standard Gaussian and then shift by the corresponding mean. When finished, your stimuli should be a single Tensor with shape (200, 25, 2), where the first dimension is over trials, the second dimension is over timesteps, and the third dimension is over stimulus features (motion and color).

Next, you will generate a context cue, which indicates to the RNN whether the motion or the color signal is relevant for its decision in a given trial. For each trial, randomly assign the context cue to be either +1 or -1 (each with probability 0.5), indicating that the relevant stimulus is motion or color, respectively. To indicate a constant context cue within each trial, replicate the cue along the time dimension of a Tensor of shape (200, 25, 1). Concatenate this with the stimulus, so that you obtain a Tensor of shape (200, 25, 3). This will be the input to your RNN.

Now, create the target outputs for the RNN. For each trial, create a running sum across the time dimension of the relevant stimulus (i.e., the motion dimension or the color dimension, depending on the context cue for that trial). The running sum represents the time-varying integrated evidence for each decision. You can compute the running sum with the `torch.cumsum` function. When finished, your target output should be a Tensor with shape (200, 25, 1).

In [5]:
condition_means = torch.tensor([-1.0, -0.5, 0.0, 0.5, 1.0])

all_stimuli = []

for mean in condition_means:
    stimuli = torch.randn(40, 25, 2) + mean
    all_stimuli.append(stimuli)

stimuli = torch.cat(all_stimuli, dim=0)

context_cues = torch.randint(0, 2, (200,)) * 2 - 1 
context_cues_expanded = context_cues.view(-1, 1, 1).expand(-1, 25, 1)

rnn_inputs = torch.cat([stimuli, context_cues_expanded], dim=-1)

relevant_indices = (context_cues == 1).long() * 0 + (context_cues == -1).long() * 1
relevant_stimulus = stimuli[torch.arange(200), :, relevant_indices]
running_sum = torch.cumsum(relevant_stimulus, dim=1)  

target_outputs = running_sum.unsqueeze(-1)

### Part 2: Implement an RNN class (5 pts)
Implement a class for constructing and running an RNN with a linear readout layer. Your class should inherit from the base class `nn.Module`. Please implement the `__init__` and `forward` methods. The `__init__` method should construct the RNN and readout layer, while the `forward` method should run the RNN and then pass its hidden states through the readout. Make sure that you return both the readout and the RNN hidden states, as we will be analyzing the latter after training the network. Note that you can directly use the RNN class presented in class.

In [3]:
class RNN(nn.Module):
  def __init__(self, in_size, hidden_size, out_size, nonlinearity='tanh'):
    super().__init__()
    self.rnn = nn.RNN(in_size, hidden_size, batch_first=True, nonlinearity=nonlinearity)
    self.decoder = nn.Linear(hidden_size, out_size)

  def forward(self, x):
    rnn_out, _ = self.rnn(x)
    batch_size, T, _ = rnn_out.shape
    rnn_out_reshaped = rnn_out.reshape(batch_size*T, -1)
    decoder_out = self.decoder(rnn_out_reshaped)
    return decoder_out.reshape(batch_size, T, -1), rnn_out

### Part 3: Implement a Dataset class (5 pts)
Implement a dataset class which inherits from the `Dataset` base class. This module should implement the `__init__`, `__len__`, and `__getitem__` methods. The `__init__` method should store the inputs and outputs (which should already be Tensors) passed via arguments to the method, the `__len__` method should return the size of the dataset, and `__getitem__` should return the datapoint at the given index passed via an argument. Note that you can use the same dataset class from the last homework assignment here, except do not wrap the inputs and outputs in a `torch.tensor`, as they are already Tensors.

In [4]:
class MyDataset(Dataset):
  def __init__(self, inputs, outputs):
    super().__init__()
    self.inputs = inputs
    self.outputs = outputs

  def __len__(self):
    return len(self.inputs)
  
  def __getitem__(self, idx):
    return self.inputs[idx], self.outputs[idx]

### Part 4: Implement the Main Training Loop (10 pts)
Implement a function which iterates through a dataset and trains the model over a specified number of epochs. In addition to what we showed in class, please add gradient clipping by norm using the `nn.utils.clip_grad_norm_` fuction and make the norm value threshold an argument to the training loop function.

As in the previous assignment, create a Python list and append the current average loss at each epoch. You can compute the average loss over all the batches for a given epoch by keeping a running sum of the loss and dividing by the # of batches, which you can get with `len(dataloader)`. Be sure to zero the running sum after each epoch. Also, make sure to append only the value of the average loss to the list using `avg_loss.item()`, which returns the value of the scalar tensor as a standard Python number. 

We also want to keep track of the average magnitude of the gradients during training, which can be an important debugging tool. To get the average gradient magnitude of your network per batch, use the following function after calling the `backward()` method on your loss:

`np.mean([p.grad.norm() for p in model.parameters()])`

Just like with the loss, we want to keep a running average of this quantity each epoch as well and append it to its own separate list. Have the function return the loss and grad_norm lists after training. Additionally, display the epoch, average loss, and average grad_norm to the console every so often, where the display frequency (or period) is an argument to the function. This can be done with a simple `print` statement.

Note that you can use basically the same training loop function from Programming Assignment 2, except you have to be careful to pass the RNN outputs to the loss function, not the hidden states or the network output tuple. Additionally, add gradient clipping by norm and make the norm value threshold an argument to the training loop.

In [18]:
def train(dataloader, net, loss_fn, optimizer, epochs, norm_threshold, print_every=1000):
    epoch_losses = []
    epoch_grads = []

    for epoch in range(epochs):
        epoch_loss = 0.0
        epoch_grad = 0.0

        for batch_inputs, batch_targets in dataloader:
            optimizer.zero_grad()

            outputs, _ = net(batch_inputs)

            loss = loss_fn(outputs, batch_targets)
            epoch_loss += loss.item()

            loss.backward()

            batch_grad_norm = np.mean([p.grad.norm().item() for p in net.parameters() if p.grad is not None])
            epoch_grad += batch_grad_norm

            torch.nn.utils.clip_grad_norm_(net.parameters(), norm_threshold)

            optimizer.step()

        avg_loss = epoch_loss / len(dataloader)
        avg_grad = epoch_grad / len(dataloader)

        epoch_losses.append(avg_loss)
        epoch_grads.append(avg_grad)

        if (epoch + 1) % print_every == 0:
            print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Grad Norm: {avg_grad:.4f}")

    return epoch_losses, epoch_grads

### Part 5: Construct the Model, Dataset, and Other Bits (5 pts)
First, create a specific RNN using our class. Then create an optimizer (use Adam this time) and pass the parameters of the model as an argument. Additionally, instantiate a loss function for our regression task (MSE loss). Finally, create a dataset using our dataset class and the data you created above. Wrap this dataset in a `DataLoader`, which will handle batching of the data. Please create an RNN which takes in 3 input, has a hidden layer of 32 neurons, has 1 output, and uses a Tanh activation function. Use a learning rate of 1e-4 and a batch size of 200.

In [19]:
rnn = RNN(3, 32, 1)
data = MyDataset(rnn_inputs, target_outputs)
dl = DataLoader(data, batch_size=200)
optimizer = optim.Adam(rnn.parameters(), lr=1e-4)
loss_fn = nn.MSELoss()

### Part 6: Train the Model (0 pts)
Train your model using your training loop for 15000 epochs and clipping the gradient norm to a value of 1. 

In [20]:
losses, grads = train(dataloader=dl, net=rnn, loss_fn=loss_fn, optimizer=optimizer, epochs=15000, norm_threshold=1, print_every=1500)

KeyboardInterrupt: 

### Part 7: Generate Average Loss Plot (10 pts)
Plot the average loss and gradient norms during training. Create two subplots with "Epoch" on the x-axis and  "Average Loss" and "Average Gradient Norm" on the y-axes, respectively. For the average loss, use a log scale for the y-axis.

### Part 8: Visualize Network Outputs (10 pts)
For one trial per stimulus-strength condition, plot the single-trial RNN readout, overlaid with the target output (1 panel per condition --> 5 panels total). To do this, you will have to run the RNN on the stimuli you select and display the RNN's readout. Note that you have to detach the output from the computational graph to plot it. This can either be done with the `detach()` function or by using the `torch.no_grad()` context. Please use a legend on the plot to indicate which is the RNN readout and which is the target output.

### Part 9: Visualize Hidden States (10 pts)
Now plot the condition-averaged activity from 3 individual hidden units (pick any unit that you like). To do this, first run the RNN $^1$ on all trials (if you haven't already stored the result above). Filter for the trials when the context cue is $+1$. Then, for each condition, compute the mean and standard deviation of the hidden states across all trials from that condition. This should involve operating on the first dimension in the Tensor containing the hidden unit activity (i.e., do not average across time steps or hidden units). For each of the hidden units you choose: create a new figure and plot the condition-averaged means and standard deviations $^2$ of the unit's activity. This should result in 5 25-timestep traces overlaid on the figure (one for each condition). Include: a title to indicate the unit #; appropriate axis labels; and a legend to indicate the conditions, labeled by the means of each stimulus distribution. 

Can you find 3 units with distinct representations?

$^1$ Note that you will again have to detach the output from the computational graph (as you will for all proceeding tasks).

$^2$ You can plot the standard deviation using the `fill_between` function in matplotlib.



### Part 10: Visualize PCA Dimensions of Hidden States (10 pts)

Now run the RNN on the entire dataset and compute the first two principle components of the hidden states. We can compute the principle components using an SVD with `torch.svd`. Note that you will have to reshape the hidden states tensor to be (-1, n_hidden), placing both the batch and sequence time step dimensions together. The SVD function should return three Tensors, which we will call `U`, `S`, and `Vh`. Take the `Vh` tensor, extract the mapping to the first two principle component directions, and then multiply this matrix with the hidden states to get our first two principle components.

For each context ($+1$ or $-1$), filter the trials and plot a separate plot. Similar to above, compute condition-average PCA trajectories: for each condition, average across all trials in that condition, preserving the time (length=25) and features (length=2) dimensions. Plot the condition-averaged PC trajectories on a 2D plot, where the X axis is the first PCA dimension, and the Y axis is the second PCA dimension. For clarity, also include a marker ('o'), which indicates the first timestep of each trajectory. As in the previous section, include a legend to indicate the conditions.

### Part 11: Identifying the dimension in the hidden states that best relates to stimulus strength (10 pts)
Finally, we will use ridge regression to identify the dimension in the space of the RNN hidden states that best predicts stimulus strength. This is similar to the Targeted Dimensionality Reduction (TDR) approach used in MS2013. 

---------------
**Ridge Regression.**
To create your regression inputs, take the hidden states of the network for the whole dataset and stack them in a (num_trials*num_timesteps, num_units) matrix. To create your regression targets, use the stimulus strengths (means of condition-specific Gaussians) that correspond to each (trial, time) in the leading dimension of the regression inputs. You can use any method you want to compute ridge regression model, but it is straightforward to implement it manually. If you implement it manually, make sure to appropriately append ones to the regression inputs to allow an offset term. The optimal parameters estimate of a ridge regression model is:

$W = (X^T X + \lambda I)^{-1} X^T Y$

Note that this is equivalent to linear regression with L2 regularization on the parameters. Use an L2 scaling of $\lambda=0.1$. Once you fit the model, extract the weights and the bias. Now apply the linear model to the hidden states, and plot the condition-averaged results versus time. 

---------------

You should produce a figure in the style of Part 9, but here the y-axis indicates activity along the stimulus-strength dimension. Again, filter for trials where context is $+1$.

### Part 12: Visualize Targeted Dimensions of Hidden States (20 pts)

Now we will attempt to recreate Fig. 5 in MS2013. We will do so by doing a simplified version of targeted dimensionality reduction (no orthogonalization). The steps are as follows:

1. Run the RNN on the entire dataset.

2. Perform ridge regression that predicts the mean values (conditions) from the hidden states for **motion context trials only**. The one-dimensional $W$ matrix found by ridge regression is the "motion axis". Project all data onto this motion axis ("motion projection"). See part 11 for details on how to do this.

3. Do the same with **color context trials only** to find the "color axis" and "color projection".

4. Perform ridge regression that predicts the choice (i.e. your target output values) from the hidden states for **all** trials. This gives the "choice axis". Again, project all data onto this axis ("choice projection").

5. Plot the following four plots on a 2x2 figure (in order, from top to bottom, left to right):

    (i) Plot motion context trials onto the motion-choice plane (x-axis is choice axis projection, y-axis is motion projection). 

        For motion context trials only, for each condition, average across the motion and choice projections. This should give you a (25,) array for each of the 5 conditions. Plot each of the arrays onto the motion-choice plane. If you want to make it look very clear and similar to Fig 5 in the paper, consider giving each line a different color or marker.

    (ii) Plot color context trials onto the motion-choice plane.

        Same as 5-(i), but changes context.

    (iii) Plot motion context trials onto the color-choice plane.

    (iv) Plot color context trials onto the color-choice plane.

### Bonus Question (2 pts)

What are a few differences between the training procedure in the paper vs the assignment?

### Administrative (2 pts)

About how many hours did you spend on this homework? There is no right or wrong answer :-)